In [6]:
#from imblearn.ensemble import RUSBoostClassifier
import sklearn as sk
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,StratifiedShuffleSplit,RepeatedKFold,GridSearchCV,RandomizedSearchCV ,StratifiedKFold,KFold
from sklearn.svm import LinearSVC , LinearSVR
from sklearn.metrics import mean_squared_error , f1_score ,accuracy_score, precision_score, recall_score, f1_score,matthews_corrcoef,auc,roc_auc_score,confusion_matrix
from sklearn.svm import SVR
from sklearn.preprocessing import scale ,StandardScaler
from sklearn.feature_selection import SelectKBest, chi2 ,f_regression
from sklearn.pipeline import Pipeline
from sklearn.metrics import  make_scorer
from sklearn.ensemble import RandomForestClassifier ,RandomForestRegressor
from xgboost import XGBClassifier ,XGBRFRegressor ,XGBRFRegressor
#from rotation_forest import RotationTreeClassifier, RotationForestClassifier
from sklearn.inspection import permutation_importance
from catboost import CatBoostClassifier,CatBoostRegressor, EShapCalcType, EFeaturesSelectionAlgorithm

import random
from sklearn.tree import DecisionTreeClassifier
from pathlib import Path
#from imblearn.over_sampling import SMOTE
import gc
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler,Normalizer,Binarizer
from datetime import datetime
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score
from catboost import Pool, CatBoostClassifier ,CatBoostRegressor
import time
from joblib import Parallel, delayed

#from lightgbm import LGBMClassifier
#from BorutaShap import BorutaShap, load_data
from sklearn.utils import shuffle
from catboost.utils import eval_metric
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import PredefinedSplit
from scipy import stats
from scipy.stats import randint
from pyts.classification import TimeSeriesForest

In [7]:
def threshold(input,value):
    model = Binarizer(threshold=value)
    input = np.array(input)
    input = input.reshape(-1,1)
    output = model.transform(input)
    return output.flatten()

def Performance(ytrue,yhat,mag,region):
    tn , fp , fn , tp = confusion_matrix(ytrue,yhat).flatten()
    mcc = matthews_corrcoef(ytrue,yhat)
    f1 = f1_score(ytrue,yhat)
    acc = accuracy_score(ytrue,yhat)
    recall = recall_score(ytrue,yhat)
    precision = precision_score(ytrue,yhat)
    output = {"tn":tn,"fp":fp,"fn":fn,"tp":tp,"mcc":mcc,"f1":f1,"acc":acc,"recall":recall,"precision":precision,"mag":mag,"region":region}
    return output

scorefunction = make_scorer(matthews_corrcoef, greater_is_better=True)




def irrel_filter(Data, target):
    L=Data.shape[0]
    MI=[]
    cols=[]
    targ_norm = (target-target.min())/(target.max()-target.min())
    targ_med=targ_norm.median()
    targ_norm_bins = (targ_norm>targ_med).astype(int)
    for i in Data.columns:
        temp_norm = (Data[i]-Data[i].min())/(Data[i].max()-Data[i].min())
        med=temp_norm.median()
        temp_norm_bins = (temp_norm>med).astype(int)
        U = 2*temp_norm_bins+targ_norm_bins
        U0 = U[U==0].shape[0]
        U1 = U[U==1].shape[0]
        U2 = U[U==2].shape[0]
        U3 = U[U==3].shape[0]
        
        p_targ_0=(U0+U2)/L
        p_targ_1=(U1+U3)/L
        p_temp_0=(U0+U1)/L
        p_temp_1=(U2+U3)/L
        
        p_targ0_temp0=U0/L
        p_targ0_temp1=U2/L
        p_targ1_temp0=U1/L
        p_targ1_temp1=U3/L
        try:
            mi=p_targ0_temp0*np.log2(p_targ0_temp0/(p_targ_0*p_temp_0))+p_targ1_temp0*np.log2(p_targ1_temp0/(p_targ_1*p_temp_0))+p_targ0_temp1*np.log2(p_targ0_temp1/(p_targ_0*p_temp_1))+p_targ1_temp1*np.log2(p_targ1_temp1/(p_targ_1*p_temp_1))
        except ZeroDivisionError:
            mi=np.nan
        if np.isnan(mi)==True:
            mi=0
        if mi<0.001:
            mi=-1
        MI.append(mi)
        if mi==-1:
            cols.append(i)
    return MI, cols

def redundant_filter(Data,MI):
    L=Data.shape[0]
    RC=np.zeros((1,(Data.shape[1]-1)))
    first=np.zeros((1,(Data.shape[1]-1)))
    second=np.zeros((1,(Data.shape[1]-1)))
    cols=[]
    for i in range(Data.shape[1]-1):
        first_var=Data.iloc[:,i]
        first_norm=((first_var-first_var.min())/(first_var.max()-first_var.min()))
        med=first_norm.median()
        first_norm_bins=(first_norm>med).astype(int)
        for j in range((i+1),(Data.shape[1]-1)):
            sec_var=Data.iloc[:,j]
            sec_norm=((sec_var-sec_var.min())/(sec_var.max()-sec_var.min()))
            med=sec_norm.median()
            sec_norm_bins=(sec_norm>med).astype(int)
            U=2*sec_norm_bins+first_norm_bins
            U0 = U[U==0].shape[0]
            U1 = U[U==1].shape[0]
            U2 = U[U==2].shape[0]
            U3 = U[U==3].shape[0]
            
            p_first_0=(U0+U2)/L
            p_first_1=(U1+U3)/L
            p_sec_0=(U0+U1)/L
            p_sec_1=(U2+U3)/L

            p_first0_sec0=U0/L
            p_first0_sec1=U2/L
            p_first1_sec0=U1/L
            p_first1_sec1=U3/L
            try:
                Mut_Inf_temp=p_first0_sec0*np.log2(p_first0_sec0/(p_first_0*p_sec_0))+p_first1_sec0*np.log2(p_first1_sec0/(p_first_1*p_sec_0))+p_first0_sec1*np.log2(p_first0_sec1/(p_first_0*p_sec_1))+p_first1_sec1*np.log2(p_first1_sec1/(p_first_1*p_sec_1))
            except ZeroDivisionError:
                Mut_Inf_temp=np.nan
            if np.isnan(Mut_Inf_temp)==True:
                Mut_Inf_temp=0
            if Mut_Inf_temp>RC[0, i]:
                RC[0, i]=Mut_Inf_temp
                first[0, i]=i
                second[0, i]=j
        if RC[0, i]>0.2:
            if MI[(first[0, i]).astype(int)]>MI[(second[0, i]).astype(int)]:
                drop_ind=second[0, i]
            else:
                drop_ind=first[0, i]
        cols.append(Data.columns[drop_ind.astype(int)])
    cols = list(set(cols))
    return cols, RC

def model_result(model,geodata,thr_mag,regionname):
    x,y = geodata.iloc[:,:-1],geodata.iloc[:,-1]
    label = threshold(y,thr_mag)
    X_train, X_test, y_train, y_test = train_test_split(x,label,test_size=0.25,shuffle=False)
    model.fit(X_train,y_train)
    yhat = model.predict(X_test)
    result = Performance(yhat,y_test,mag=thr_mag,region=regionname)
    print(result)
    return result
    
def model_result_mrmr(model,geodata,thr_mag,regionname):
    x,y = geodata.iloc[:,:-1],geodata.iloc[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(x,y,test_size=0.25,shuffle=False)

    x,y = X_train.iloc[:,:-1],X_train.iloc[:,-1]
    MI1,cols = irrel_filter(X_train,y_train)
    cols2,RC = redundant_filter(X_train,MI=MI1)
    coldrop = cols2+cols

    X_train.drop(coldrop,axis=1,inplace=True)
    X_test.drop(coldrop,axis=1,inplace=True)

    y_train = threshold(y_train,thr_mag)
    y_test = threshold(y_test,thr_mag)
    
    model.fit(X_train,y_train)
    yhat = model.predict(X_test)
    result = Performance(yhat,y_test,mag=thr_mag,region=regionname)
    print(result)
    return result

In [8]:
os.chdir(r"C:\Users\12105\Desktop\HZ with gap\raw_data")

In [12]:
sc_data = pd.read_csv("SC.csv")
hindu_data = pd.read_csv("H.csv")
sc_data.drop_duplicates(keep= "first",inplace=True)

In [86]:
len(sc_data)*0.7

20387.5

In [106]:
def model_result(model,geodata1,thr_mag,regionname):
    geodat2 = geodata1.replace(np.inf,np.nan)
    geodata = geodat2.dropna()
    x,y = geodata.iloc[:,:-1],geodata.iloc[:,-1]
    label = threshold(y,thr_mag)
    X_train, X_test, y_train, y_test = train_test_split(x,label,test_size=0.3,shuffle=False)
    model.fit(X_train,y_train)
    yhat = model.predict(X_test)
    result = Performance(yhat,y_test,mag=thr_mag,region=regionname)
    print(result)
    return result

In [16]:
model_result(TimeSeriesForest(n_jobs=-1),hindu_data,6,"sc")

{'tn': 1006, 'fp': 38, 'fn': 44, 'tp': 0, 'mcc': -0.039054693846911764, 'f1': 0.0, 'acc': 0.9246323529411765, 'recall': 0.0, 'precision': 0.0, 'mag': 6, 'region': 'sc'}


{'tn': 1006,
 'fp': 38,
 'fn': 44,
 'tp': 0,
 'mcc': -0.039054693846911764,
 'f1': 0.0,
 'acc': 0.9246323529411765,
 'recall': 0.0,
 'precision': 0.0,
 'mag': 6,
 'region': 'sc'}